# Creating Web Maps using Folium

Folium is a powerful Python library that helps you create several types of Leaflet maps. By default, Folium creates a map in a separate HTML file. Since Folium results are interactive, this library is very useful for dashboard building. You can also create inline Jupyter maps in Folium

https://python-visualization.github.io/folium/



In [ ]:
import folium

In [ ]:
m = folium.Map(location=[47.37825,8.5367835], zoom_start=18) # Zürich, Switzerland
m

In [ ]:
m = folium.Map(location=[47.37825,8.5367835],
           tiles='Stamen Toner',   # "OpenStreetMap", "Stamen Terrain", "Stamen Toner"
           zoom_start=14)
m

### Adding Markers

In [ ]:
m = folium.Map(location=[47.37825,8.5367835], zoom_start=16)

folium.Marker([47.37695,8.5387885], popup="Hotel <b>Schweizerhof</b><br/><br/>This hotel is located in the center").add_to(m)
folium.Marker([47.376386,8.5386506], popup="Hotel St. Gotthard").add_to(m)
m

### Using Markers with FontAwesome ("fa")

https://fontawesome.com/icons?d=gallery&m=free

In [ ]:
m = folium.Map(location=[47.37825,8.5367835], zoom_start=16)

folium.Marker([47.37695,8.5387885], 
              popup="Hotel Schweizerhof",
              icon=folium.Icon(color="red", prefix="fa", icon="hotel")).add_to(m)

folium.Marker([47.376386,8.5386506], 
              popup="Hotel St. Gotthard",
              icon=folium.Icon(color="green", prefix="fa", icon="hotel")).add_to(m)

folium.Marker([47.376192, 8.540005], 
              popup="Hotel Townhouse", 
              icon=folium.Icon(color="blue", prefix="fa", icon="beer")).add_to(m)

m

### Saving Maps as HTML

Example: Mountain Tops

In [ ]:
m = folium.Map(location=[45.922513343092916, 7.835574679184418], zoom_start=9)

folium.Marker([45.922513343092916, 7.835574679184418], popup='Liskamm: 4527m').add_to(m)
folium.Marker([45.941997570720375, 7.869820276613906], popup='Nordend: 4609m').add_to(m)
folium.Marker([46.10902325837147, 7.863895545667632], popup='Nadelhorn: 4327m').add_to(m)
folium.Marker([45.932186337151684, 7.8714190183674555], popup='Zumsteinspitze: 4563m').add_to(m)
folium.Marker([46.08336532442726, 7.857296913890337], popup='Täschhorn: 4491m').add_to(m)
folium.Marker([45.91669904679932, 7.863563975062021], popup='Ludwigshöhe: 4341m').add_to(m)
folium.Marker([45.93756139078208, 7.299279971077615], popup='Grand Combin de Grafeneire: 4314m').add_to(m)
folium.Marker([45.922513343092916, 7.835574679184418], popup='Lyskamm: 4527m').add_to(m)
folium.Marker([45.93683662540408, 7.866814344981748], popup='Dufourspitze (Pointe Dufour): 4634m').add_to(m)
folium.Marker([46.10129664518156, 7.716156885858494], popup='Weisshorn: 4506m').add_to(m)
folium.Marker([45.976340506120614, 7.658691510512221], popup='Monte Cervino: 4478m').add_to(m)
folium.Marker([45.976340506120614, 7.658691510512221], popup='Matterhorn: 4478m').add_to(m)
folium.Marker([45.93674004101607, 7.86855410887458], popup='Grenzgipfel: 4618m').add_to(m)
folium.Marker([45.92712756883081, 7.876921984235257], popup='Signalkuppe (Punta Gnifetti): 4554m').add_to(m)
folium.Marker([46.093839189553464, 7.858928716434883], popup='Dom: 4545m').add_to(m)
folium.Marker([46.107109586833495, 7.711724522200983], popup='Grand Gendarme: 4331m').add_to(m)
folium.Marker([45.919638502715564, 7.8711910872756405], popup='Parrotspitze: 4432m').add_to(m)
folium.Marker([46.093839189553464, 7.858928716434883], popup='Mischabel: 4545m').add_to(m)
folium.Marker([46.03426257063022, 7.61204033560156], popup='Dent Blanche: 4357m').add_to(m)

m.save("mountain_tops.html")
m

## GeoPandas Geometry to Folium Map

We already downloaded the natural earth dataset in lesson 3, if you skipped it somehow, I added the code to reinstall it...

In [ ]:
import geoutils
import zipfile
import os

if not os.path.exists("geodata"):
    os.mkdir("geodata")

if not os.path.exists("geodata/packages/natural_earth_vector.gpkg"):
    ne = geoutils.geodata["natural-earth"] # URL to Natural Earth Dataset (GeoPackage Format)
    geoutils.download(ne, "geodata/ne.gpkg.zip")
else:
    print("Dataset is already downloaded")
    
if not os.path.exists("geodata/packages/natural_earth_vector.gpkg"):
    with zipfile.ZipFile("geodata/ne.gpkg.zip", 'r') as z:
        z.extractall("geodata")

In [ ]:
import geopandas as gpd
%matplotlib inline


gdfCountries = gpd.read_file("geodata/packages/natural_earth_vector.gpkg", 
                              layer="ne_10m_admin_0_countries", 
                              encoding="utf-8")


gdfQuery = gdfCountries.query("NAME == 'France'")
jsondata = gdfQuery.to_json()

In [ ]:
jsondata;

In [ ]:
m = folium.Map(location=[47.37825,8.5367835], zoom_start=6)

folium.GeoJson(
    jsondata,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

m

## Adding OpenStreetMap Data


In [ ]:
%matplotlib inline

import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ox.config(use_cache=True, log_console=False)

### Retrieve Street Network from OSM Data

* drive - get drivable public streets (but not service roads)
* drive_service - get drivable streets, including service roads
* walk - get all streets and paths that pedestrians can use (this network type ignores one-way directionality)
* bike - get all streets and paths that cyclists can use
* all - download all non-private OSM streets and paths
* all_private - download all OSM streets and paths, including private-access ones

In [ ]:
place = "Basel, Switzerland"
graph = ox.graph_from_place(place, network_type='drive')

In [ ]:
fig, ax = ox.plot_graph(graph)

#### Save Graph as ESRI Shapefile

In [ ]:
ox.save_graph_shapefile(graph, filepath='network-shape')

In [ ]:
nodes, streets = ox.graph_to_gdfs(graph)

In [ ]:
len(streets)

In [ ]:
streets.head()

In [ ]:
street_types = pd.DataFrame(streets["highway"].apply(pd.Series)[0].value_counts().reset_index())
street_types.columns = ["type", "count"]

In [ ]:
street_types

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
sns.barplot(y="type", x="count", data=street_types, ax=ax)
plt.tight_layout()
plt.show();

### Putting the Street Network on a Folium Map

In [ ]:
m = folium.Map([47.561086, 7.588686],
zoom_start=13,tiles="CartoDB dark_matter")

jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}
folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)
m

### Routing

There are two train stations in Basel, lets find the route between them:

In [ ]:
train_station_1 = ox.geocode('Badischer Bahnhof, Basel Switzerland')
train_station_1

In [ ]:
train_station_2 = ox.geocode('Bahnhof SBB, Basel, Switzerland')
train_station_2

#### Let's add them as POI to the map **above**:

In [ ]:
m = folium.Map([47.561086, 7.588686], zoom_start=13,tiles="CartoDB dark_matter")

#jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}
folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)

folium.Marker(train_station_1, 
              popup="Badischer Bahnhof",
              icon=folium.Icon(color="green", prefix="fa", icon="train")).add_to(m)

folium.Marker(train_station_2, 
              popup="Bahnhof SBB",
              icon=folium.Icon(color="red", prefix="fa", icon="train")).add_to(m)
m

In [ ]:
origin_node = ox.get_nearest_node(graph, train_station_1)
destination_node = ox.get_nearest_node(graph, train_station_2)

#### Find the route between the nodes

Using NetworkX module to calculate

In [ ]:
route = nx.shortest_path(graph, origin_node, destination_node)

In [ ]:
fig, ax = ox.plot_graph_route(graph, route)

In [ ]:
from shapely.geometry import LineString

line = []
for i in route:
    point = (graph.nodes[i]['x'],graph.nodes[i]['y'])
    line.append(point)
    print(point)
    
ls = LineString(line)

#### Create a GeoDataFrame from the LineString (optional)


In [ ]:
gdf_route = pd.DataFrame(data= [['route 1',ls]],columns=['route','geometry'])
gdf_route

In [ ]:
gdf_route = gpd.GeoDataFrame(gdf_route,geometry='geometry')
gdf_route

In [ ]:
gdf_route.plot();

Convert this route to GeoJSON

In [ ]:
jsonroute = gdf_route.to_json()

In [ ]:
m = folium.Map([47.561086, 7.588686], zoom_start=13,tiles="CartoDB dark_matter")

#jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}

route_style = {'color': '#FF0000',
              'weight' : '4'}

folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)
folium.GeoJson(jsonroute, style_function=lambda x: route_style).add_to(m)

folium.Marker(train_station_1, 
              popup="Badischer Bahnhof",
              icon=folium.Icon(color="green", prefix="fa", icon="train")).add_to(m)

folium.Marker(train_station_2, 
              popup="Bahnhof SBB",
              icon=folium.Icon(color="red", prefix="fa", icon="train")).add_to(m)
m

### Building Footprints

In [ ]:
buildings = ox.geometries_from_place("Basel, Switzerland", tags={'building':True})

buildings.shape

In [ ]:
buildings.head()

In [ ]:
ox.plot_footprints(buildings, figsize=(16,15));

### Retrieving Buildings

In [ ]:
museum = buildings.query("tourism == 'museum'")
museum = museum[['name', 'geometry']]
museum.head(20)

In [ ]:
ox.plot_footprints(museum, figsize=(16,15));

### Putting those Buildings on a Folium Map

In [ ]:
import html

m = folium.Map([47.561086, 7.588686],zoom_start=15)


style = {'color': '#FF0000',
         'fillColor': '#FFFF00',
         'weight':'2'
        }

def building(building):
    jsondata = gpd.GeoSeries([building["geometry"]]).to_json()
    name = html.escape(building["name"]) 
    folium.GeoJson(jsondata, style_function=lambda x: style).add_child(folium.Popup(name)).add_to(m)

museum.apply(building, axis=1)

m.save("museums.html")
m

### Creating POI out of buildings

In [ ]:
def get_geometry(x):
    g = x["geometry"]
    c = g.centroid
    return c
   
centroid = museum.apply(get_geometry, axis=1)

In [ ]:
museum["centroid"] = centroid 

In [ ]:
museum.head()

In [ ]:
import html

m = folium.Map([47.561086, 7.588686],zoom_start=15)


def building(building):
    lng = building["centroid"].x
    lat = building["centroid"].y

    name = html.escape(building["name"])
   
    folium.Marker([lat,lng], 
              popup=name,
              icon=folium.Icon(color="red", prefix="fa", icon="university")).add_to(m)


museum.apply(building, axis=1)

m

### Retrieve POI from OSM

In [ ]:
atm = ox.pois_from_place("Basel, Switzerland", tags={'amenity': 'atm'})

In [ ]:
len(atm)

In [ ]:
atm.head()

In [ ]:
import html

m = folium.Map([47.561086, 7.588686],zoom_start=15)


def atm2poi(x):
    lng = x["geometry"].x
    lat = x["geometry"].y
    operator = x["operator"]

    if type(operator) == str:
        name = html.escape(operator)
    else:
        name = "unkown bank"
   
    folium.Marker([lat,lng], 
              popup=name,
              icon=folium.Icon(color="green", prefix="fa", icon="credit-card")).add_to(m)


atm.apply(atm2poi, axis=1)

m